##  <center>Типы данных в PostgreSQL<center>

**SQL** — это язык со строгой типизацией, в котором каждый элемент данных имеет некоторый тип, определяющий его поведение и допустимое использование.

Типы данных в PostgreSQL можно разделить на несколько групп. К основным относятся:

* числовые типы — для хранения чисел (целых и дробных);
* типы даты/времени — для хранения даты, времени, часовых поясов;
* символьные типы — для хранения символов или строк;
* логический тип — для хранения значений типа «истина», «ложь».

 Каждая группа (кроме логического типа) объединяет несколько типов, отличающихся по допустимому диапазону хранимых данных. Например, числовые типы данных хранят целые числа, дробные числа, а строковые — подразделяются на типы с фиксированной и переменной длиной. А скажем, целочисленный тип данных integer может хранить значения в диапазоне от -2147483648 до 2147483647.

 Кроме возможных значений из доступного диапазона, поле независимо от выбранного типа может принимать значение **NULL** (отсутствующее значение).

### <center>Основные типы дат<center>

Любой анализ событий во времени подразумевает работу с датой или временем. Для них в Postgres существует несколько типов данных — все они представлены в таблице ниже.

![](images/data_types.jpg)

#### <center>Тип **Timestamp**<center>

**Timestamp** — наиболее распространённый тип данных, так как он содержит и дату, и время, а также используется в любых логах событий, временных рядах и в большинстве системных таблиц.

Согласно стандарту ISO, значение выглядит как "2019-07-14 01:35:44.702165+00", где перечислены через точку год-месяц-день, время и часовой пояс.

Для получения текущего значения даты и времени в Postgres используются функции ***CURRENT_TIMESTAMP*** (есть в стандарте SQL) и ***NOW()*** (есть в большинстве баз данных)

#### <center>Тип **Timestamp with Time Zone**<center>

**Timestamp with time zone** позволяет хранить сведения о часовом поясе, что может быть удобно при анализе географически распределённых временных данных для единообразия хранения.

Список часовых поясов можно увидеть в системном справочнике **pg_timezone_names**.

![](images/pg_timezone.jpg)

Описание этого справочника вы не сможете посмотреть в Metabase, но можете к нему обратиться, написав SQL-запрос.

*Запрос:*

**SELECT** *  
**FROM** pg_timezone_names

*Запрос:*

**SELECT NOW()** at time zone 'Europe/Moscow'

Указание **at time zone** позволяет переводить дату/время без часового пояса в дату/время с часовым поясом и обратно, а также пересчитывать значения времени для различных часовых поясов.

В таблице ниже приведены примеры того, как работает **at time zone** для разных типов данных.

![](images/at_time_zone.jpg)

#### **Задание 3.1**

Давайте узнаем, сколько сейчас времени в другом регионе, например Лос-Анджелесе. Напишите запрос, который выведет текущие время и дату в часовом поясе Лос-Анджелеса ("America/Los_Angeles"). Столбец в выдаче — now (время и дата в нужном часовом поясе).

*Запрос:*

**SELECT NOW()** at time zone 'America/Los_Angeles' now

### <center>Тип **Date**<center>

С типом date вы уже знакомы, его реализация предельно проста. Отметим только, что тип timestamp (with/without time zone) можно легко перевести в соответствующую дату, используя синтаксис

***"timestamp_column"::date***

И наоборот, тип date преобразуется в timestamp (дата и 00:00:00) с помощью

***"date_column"::timestamp***  

Для получения текущей даты можно использовать

**SELECT** CURRENT_DATE

или

**SELECT** now()::date

#### **Задание 3.2**

Предположим, у нас есть дата и время какого-то события и мы хотим посмотреть, к какой дате оно относится для Москвы и для UTC. Используйте следующий подзапрос и выведите дату в ts в Московском часовом поясе и в поясе UTC:


with x as 
(select '2018-12-31 21:00:00+00'::timestamp with time zone ts)  

Столбцы в выдаче: dt_msk (дата в московском часовом поясе), dt_utc (дата в UTC).  
Примечание: в данном куске SQL-кода мы обозначаем результат запроса SQL как таблицу с именем x. В этой таблице содержится столбец ts. Воспользуйтесь этой таблицей для решения задачи. Например, чтобы посмотреть всё содержимое таблицы x, достаточно написать следующий запрос:

with x as 
(
select '2018-12-31 21:00:00+00'::timestamp with time zone ts
)  
select * from x

*Запрос:*

with x as  
&ensp;&ensp;&ensp;&ensp;(select '2018-12-31 21:00:00+00'::timestamp with time zone ts)
    
**SELECT**  
&ensp;&ensp;&ensp;&ensp;(ts at time zone 'Europe/Moscow')::date dt_msk,  
&ensp;&ensp;&ensp;&ensp;(ts at time zone 'UTC')::date dt_utc  
**FROM** x


### <center>Тип **Interval**<center>

**Interval** — тип данных, позволяющий хранить разницу между двумя временными метками. 

Интервалы хранят данные в трёх отдельных полях — месяцах, днях, секундах. Это сделано из-за того, что количество дней в месяце и часов в дне может быть разным. Пример значения такого типа: "195 days -10:52:23.563955".

### <center>Функции для работы с типами дат<center>

#### <center>**Функция EXTRACT()**<center>

Функция **extract()** получает из значений даты/времени такие поля, как год или час.  
Здесь источник — значение типа *timestamp, time* или *interval*. Допускается и тип *date*, поскольку он приводится к типу *timestamp*.  
Указанное поле представляет собой идентификатор, по которому из источника выбирается заданное поле. Функция **extract()** возвращает значения типа *double precision*.  

#### **DAY**

Для значений *timestamp* это день месяца (1-31), для значений *interval* — число дней.

**SELECT** EXTRACT(**DAY FROM** TIMESTAMP '2001-02-16 20:38:40');  
Результат: 16 

**SELECT** EXTRACT(**DAY FROM** INTERVAL '40 days 1 minute');  
Результат: 40

#### **HOUR**

Час (0-23).

**SELECT** EXTRACT(**HOUR FROM** TIMESTAMP '2001-02-16 20:38:40');  
Результат: 20

#### **MONTH**

Номер месяца, считая с января (1) до декабря (12).

**SELECT** EXTRACT(**MONTH FROM** TIMESTAMP '2001-02-16 20:38:40');  
Результат: 2

#### **YEAR**

Поле года. Учтите, что года 0 не было, и это следует иметь в виду, вычитая из годов нашей эры годы до нашей эры.

**SELECT** EXTRACT(**YEAR FROM** TIMESTAMP '2001-02-16 20:38:40');  
Результат: 2001

#### **ISOYEAR**

Год по недельному календарю *ISO 8601*, в который попадает дата (не применимо к интервалам).

**SELECT** EXTRACT(**ISOYEAR FROM** DATE '2006-01-01');  
Результат: 2005

**SELECT** EXTRACT(**ISOYEAR FROM** DATE '2006-01-02');  
Результат: 2006

Год по недельному календарю *ISO* начинается с понедельника недели, в которой оказывается 4 января, так что в начале января или в конце декабря год по *ISO* может отличаться от года по григорианскому календарю. Подробнее об этом рассказывается в описании поля *week*

#### **WEEK**

Номер недели в году по недельному календарю *ISO 8601*. По определению, недели *ISO 8601* начинаются с понедельника, а первая неделя года включает 4 января этого года. Другими словами, первый четверг года всегда оказывается в первой неделе этого года.

В системе нумерации недель *ISO* первые числа января могут относиться к 52-й или 53-й неделе предыдущего года, а последние числа декабря — к первой неделе следующего года. Например, 2005-01-01 относится к 53-й неделе 2004 г., а 2006-01-01 — к 52-й неделе 2005 г., тогда как 2012-12-31 включается в первую неделю 2013 г. Поэтому для получения согласованных результатов рекомендуется использовать поле isoyear в паре с week.

**SELECT** EXTRACT(**WEEK FROM** TIMESTAMP '2001-02-16 20:38:40');  
Результат: 7

#### **CENTURY**

**SELECT** EXTRACT(**CENTURY FROM** TIMESTAMP '2000-12-16 12:21:13');  
Результат: 20

**SELECT** EXTRACT(**CENTURY FROM** TIMESTAMP '2001-02-16 20:38:40');  
Результат: 21

Первый век начался 0001-01-01 00:00:00, хотя люди в то время так и не считали. Это определение распространяется на все страны с григорианским календарём.Века с номером 0 не было; считается, что 1 наступил после -1

#### **DECADE**

Десятилетие.

**SELECT** EXTRACT(**DECADE FROM** TIMESTAMP '2001-02-16 20:38:40');  
Результат: 200

#### **EPOCH**

Для значений *timestamp with time zone* это число секунд с 1970-01-01 00:00:00 UTC (может быть отрицательным); для значений *date* и *timestamp* это число секунд с 1970-01-01 00:00:00 по местному времени, а для *interval* — общая длительность интервала в секундах.

**SELECT** EXTRACT(**EPOCH FROM** TIMESTAMP **WITH* TIME ZONE '2001-02-16 20:38:40.12-08');  
Результат: 982384720.12

**SELECT** EXTRACT(**EPOCH FROM** INTERVAL '5 days 3 hours');  
Результат: 442800

Преобразовать время эпохи обратно, в значение дата/время, с помощью *to_timestamp* можно так:

**SELECT** to_timestamp(982384720.12);  
Результат: 2001-02-17 04:38:40.12+00

#### **DOW**

День недели, считая с воскресенья (0) до субботы (6).

**SELECT** EXTRACT(**DOW FROM** TIMESTAMP '2001-02-16 20:38:40');  
Результат: 5

Заметьте, что в *extract()* дни недели нумеруются не так, как в функции *to_char(..., 'D')*.

#### **DOY**

День года (1-365/366).

**SELECT** EXTRACT(**DOY FROM** TIMESTAMP '2001-02-16 20:38:40');  
Результат: 47

#### **ISODOW**

День недели, считая с понедельника (1) до воскресенья (7).

**SELECT** EXTRACT(**ISODOW FROM** TIMESTAMP '2001-02-18 20:38:40');  
Результат: 7

Результат отличается от dow только для воскресенья. Такая нумерация соответствует *ISO 8601*.

#### **Задвние 4.1**

Давайте посчитаем помесячную статистику по доставкам, используя функцию extract. Напишите запрос, который выведет год, месяц и количество доставок. Отсортируйте по году и по месяцу в порядке возрастания. Столбцы в выдаче: year_n (номер года), month_n (номер месяца), qty (количество доставок).

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;**EXTRACT**(**YEAR FROM** s.ship_date) year_n,  
&ensp;&ensp;&ensp;&ensp;**EXTRACT**(**MONTH FROM** s.ship_date) month_n,  
&ensp;&ensp;&ensp;&ensp;**COUNT**(s.ship_id) qty  
**FROM** sql.shipment s  
**GROUP BY** year_n, month_n  
**ORDER BY** year_n, month_n


### <center>**Функция to_char()**<center>

Функция *to_char()* нужна для форматирования даты времени и интервалов в нужный текст.  
Например, вы хотите вывести год, месяц и день со специфическим разделителем или получить текстовое наименование месяца или дня недели. По результату работы она очень близка к extract(), но больше нацелена именно на форматирование. Ниже вы видите таблицу с примерами вызовов:

![](images/to_char.jpg)

Первый параметр для функции — дата, время или интервал, а второй — текстовая маска получаемого результата. Перечень значений, которые можно использовать для маски, представлен в таблице ниже.  

![](images/char_mask.jpg)

Любой текст, который относится к кодам форматирования, будет сохранён как есть. Чтобы оставлять какие-то подсказки и использовать обычные буквы, которые не встречаются в таблице выше, необходимо заключить текст, который нужно оставить без изменений, в **двойные кавычки**.  
Предположим, мы хотим вывести сегодняшнюю дату в формате *"Hello! Today is #название дня недели год.название месяца.день#"* текстом. Для этого нужно выполнить следующий код: 

**select to_char**(now(),'"Hello! Today is" DAY yyyy-Mon-dd')

#### **Задание 4.2**

Давайте выведем текст текущего времени для сервиса точного времени. Напишите запрос, который выводит текст "Точное время x часов y минут z секунд" (текст в кавычки заключать не нужно), где x, y, z — часы, минуты и секунды соответственно, при условии, что сообщение нужно вывести для московского часового пояса. Время введите в 24-часовом формате. Столбцы в выдаче: msg (сообщение).

*Запрос:*

**SELECT** to_char(**NOW()** at time zone 'Europe/Moscow', '"Точное время" HH24 "часов" MI "минут" SS "секунд"') msg

### <center>**Функция date_trunc()**<center>

Функция date_trunc() позволяет отсечь заданное время, дату или дату со временем до нужной точности.  
Формат вызова:

 date_trunc('поле', значение) 

Например, если мы хотим округлить текущее время-дату до минут, то можно вызвать

**select date_trunc**('minute',now())

Для получения разной степени точности вместо minute можно использовать следующие параметры:

* microseconds;
* milliseconds;
* second;
* minute;
* hour;
* day;
* week;
* month;
* quarter;
* year;
* decade;
* century;
* millennium.

Из их названий легко понять, какая временная единица подразумевается.

#### **Задание 4.3**

Давайте подготовим данные для квартальной отчётности компании. Напишите запрос, который выведет дату доставки, округлённую до квартала, и общую массу доставок. Отсортируйте по кварталу в порядке возрастания. Столбцы в выдаче: q (начало квартала, тип date), total_weight (сумма масс доставок за квартал).

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;date_trunc('quarter', s.ship_date)::date q,  
&ensp;&ensp;&ensp;&ensp;**SUM**(s.weight) total_weight  
**FROM** sql.shipment s  
**GROUP BY** q  
**ORDER BY** q

### <center>**Математические операторы**<center>

К любой дате можно прибавить (и вычесть из неё) целое число X и получить другую дату, которая больше (меньше) изначальной.

Пример:

**SELECT** '2019-01-01'::date + 10  
Результат: '2019-01-11'

Это — дата на 10 дней позже 2019-01-01.

При добавлении (или вычитании) целого числа к дате Postgres учитывает переходы между месяцами и годами и даёт верный ответ, соответствующий календарю. Учитываются даже високосные годы.

Пример:

**SELECT** '2019-01-01'::date + 500  
Результат: '2020-05-15'

Как видим, сменились и год, и месяц, и день.

Аналогично можно вычесть из одной даты другую и получить расстояние в днях между этими датами. При такой операции тоже будет честная разница по календарю.

Пример:

**SELECT** '2019-02-10'::date - '2017-03-01'::date    
Результат: 711

#### **Задание 4.4**

Давайте оценим, в каком интервале совершались доставки в разных городах. Напишите запрос, который выведет разницу между последним и первым днём доставки по каждому городу. Отсортируйте по первому и второму столбцам. Столбцы в выдаче: city_name (название города) и days_active (время от первой до последней доставки в днях).

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;c.city_name city_name,   
&ensp;&ensp;&ensp;&ensp;**MAX**(s.ship_date)::date-**MIN**(s.ship_date)::date days_active  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.shipment s  
&ensp;&ensp;&ensp;&ensp;**JOIN** sql.city c **ON** s.city_id = c.city_id  
**GROUP BY** c.city_id  
**ORDER BY** 1, 2

### <center>Строковые данные: основные типы<center>

В Postgres есть три основных типа данных для работы со строками: **character**, **character varying** и **text**.

#### **CHARACTER**

Cтрока фиксированной длины, дополненная пробелами. Длина строки такого типа всегда одинакова и задаётся в скобках.

Например, в столбце *character(5)* всегда будет пять символов: строку большей длины туда вставить не получится, а строка меньшей длины будет дополняться ведущими пробелами. Слово "SQL" в таком столбце будет выглядеть как "  SQL".

Основной паттерн использования такого типа — универсальные справочники буквенных кодов, например код страны в стандарте ISO (RU, US, UK и т. д.)

#### **CHARACTER VARYING**

Строка ограниченной переменной длины.

Например, в столбце типа *character varying(5)* нельзя будет хранить строку большей длины, но могут быть любые строки с меньшей длиной.

Этот тип данных повсеместно используется для хранения данных, поскольку позволяет ограничить ввод, сохраняя при этом возможность иметь строки произвольной длины

#### **TEXT**

Cтрока неограниченной длины.

Самый удобный тип для пользователя, но самый тяжеловесный для администратора баз данных, так как в строку можно записать любой текст.

Для удобства все текстовые поля в нашем датасете с доставками представлены типом *text*.

### <center>Функции и операторы для работы со строками<center>

#### <center>**Операторы**<center>

#### **Соединение строк**

Оператор конкатенации строк — **||** (две вертикальные черты). Он позволяет объединять две и более строки.  
Конструкции с оператором соединения строк записываются следующим образом:

**строка1 || строка2 || ... || строкаN**  

**Важно!** Результатом соединения любых типов строковых данных будет тип text.  
Напишем запрос, который позволит подготовить простые select-запросы для всех таблиц из схемы.

**SELECT** 'select * from '||t.table_schema||'.'||t.table_name||';' query  
**FROM** information_schema.tables t  
**WHERE** table_schema = 'shipping' 

В результате должно получиться пять SQL-запросов, по одному к каждой таблице из схемы *shipping*. Как мы видим, соединять можно и рукописный текст, и значения столбцов в любом произвольном порядке.  
**Важно!** Если вы соединяете любую строку и NULL, то результатом будет NULL. Поэтому, если вы формируете какой-то текст на основе поля, в котором присутствует NULL, используйте оператор *coalesce*.

#### **Задание 6.1**

Составим текстовый шаблон сообщения о доставке по конкретному водителю для наших клиентов. Напишите SQL-запрос, который выведет следующее сообщение для каждого водителя по форме:

*Ваш заказ доставит водитель #Имя Фамилия#. Его контактный номер: #Номер#*

Где #Имя Фамилия# и #Номер# взяты из справочника водителей. Если номер не указан, то выведите прочерк (-). Для номеров рекомендуем использовать COALESCE. Пример из таблицы для наглядности:

*Ваш заказ доставит водитель Adel Al-Alawi. Его контактный номер: (901) 947-4433*

Столбец к выдаче — msg (текст сообщения).

*Запрос:*

**SELECT** 'Ваш заказ доставит водитель '||d.first_name||' '||d.last_name||'. Его контактный номер: '||**COALESCE**(d.phone, '-') msg  
**FROM** sql.driver d

#### <center>**Функции**<center>

#### **Функции upper() и lower()**

Функции **upper(your_text)** и **lower(your_text)** переводят каждый символ вашего текста в верхний и нижний регистр соответственно.  
Пример:

**SELECT upper**('Abc') s1 ,**lower**('xYz') s2  

Результат функций **upper()** и **lower()** — тоже строковый, а значит, к нему можно применять все функции, применимые к этому типу данных.

#### **Задание 6.2**

Cоставим справочник названий клиентов, у которых более десяти доставок. Данные сохраним в нижнем регистре, чтобы передавать их в другие системы (например, для обзвона), которые не чувствительны к регистру. Напишите запрос, который выводит все id названий клиентов, у которых более десяти доставок, в нижнем регистре. Отсортируйте результат по cust_id в порядке возрастания. Столбцы в выдаче: cust_id (id клиента) и cust_name (название клиента в нижнем регистре).

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;cs.cust_id cust_id,  
&ensp;&ensp;&ensp;&ensp;**lower**(cs.cust_name) cust_name  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.customer cs  
&ensp;&ensp;&ensp;&ensp;**JOIN** sql.shipment s **ON** cs.cust_id = s.cust_id  
**GROUP BY** cs.cust_id  
**HAVING COUNT**(s.ship_id)>10  
**ORDER BY** cust_id

#### **Функция replace()**

Запись строится следующим образом:

**replace(string text, from text, to text)**

Эта запись означает, что в исходной строке *string* мы заменяем все вхождения строки *from* на строку *to*.  
Разберём на примере.

**SELECT replace**('малако','а','о')  

Результат выполнения такого запроса будет молоко, т. е. все буквы «а» в строке «малако» были заменены на «о».  
Результат функции **replace()** — строка, а значит, к ней тоже можно применять все известные нам функции работы со строками.  
Если вы хотите удалить из строки какие-то символы, то третьим параметром (to) передайте пустую строку ''(одинарные кавычки без символа внутри).  
Например, сделаем из строки *"Hello, world!"* строку *"Hello!"*.

**SELECT replace**('Hello, world!',', world','')

#### **Задание 6.3**

Составим справочник utm-меток, для того чтобы передавать город и штат прямо в адресной строке. (Если вы не знаете, что такое utm-метка, почитайте статью на Вики. К программе курса это не относится, но знать полезно.) Напишите SQL-запрос, который выведет список сочетаний из справочника следующего вида: *название_штата__название_города*, где названия штата и города взяты из справочника городов и переведены в нижний регистр. Столбец к выдаче — *utm* (форматированный штат-город). Отсортируйте полученный справочник по алфавиту. Обратите внимание! Все пробелы в названиях городов и штатов замените символом *'_'* (одно нижнее подчёркивание), а для разделения названий города и штата используйте *'__'* (два последовательных нижних подчёркивания). Пример из таблицы для наглядности: *new_jersey__union_city*

*Запрос:*

**SELECT lower**(**replace**(c.state,' ','_'))||'__'||**lower**(**replace**(c.city_name,' ','_')) utm  
**FROM** sql.city c  
**ORDER BY** utm

#### **Функции left() и right()**

Функции **left(string,n)** и **right(string,n)** оставляют **n** левых или правых символов от строки, поданной на вход. Давайте разобьём строку 'Один два три' на слова, используя эти функции.

*Запрос:*

with t as  
&ensp;&ensp;&ensp;&ensp;(**SELECT** 'Один два три'::text sample_string)  
**SELECT**   
&ensp;&ensp;&ensp;&ensp;**left**(t.sample_string,4) one,   
&ensp;&ensp;&ensp;&ensp;**right**(**left**(t.sample_string,8),3) two,  
&ensp;&ensp;&ensp;&ensp;**right**(t.sample_string,3) three  
**FROM** t  

Пример:

**SELECT left**('0123456789', - 2), **right**('0123456789', - 2)

Результат: 01234567 и 23456789 (в первом случае — восемь символов с «отрезанными» 89 и во втором случае — восемь символов с «отрезанными» 01

#### **Задание 6.4**

Представим, что к вам пришёл разработчик, который хочет сократить поле state в таблице city до четырёх символов, и попросил проверить, останeтся ли значения в нём уникальными. Чтобы ответить на этот вопрос, напишите SQL-запрос, который выведет первые четыре символа названия штата и количество уникальных названий штатов, которому они соответствуют. Оставьте только те, которые относятся к двум и более штатам. Добавьте сортировку по первому столбцу. Столбцы в выдаче: code (четыре первых символа в названии штата), qty (количество уникальных названий штата, начинающихся с этих символов).

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;**DISTINCT left**(c.state,4) code,  
&ensp;&ensp;&ensp;&ensp;**COUNT**(**DISTINCT** c.state) qty  
**FROM** sql.city c  
**GROUP BY** code  
**HAVING COUNT**(**DISTINCT** c.state) > 1  
**ORDER BY** code

#### **Функция format()**

Функция **format()** используется для составления форматированного текста с подстановками. То же самое можно сделать через конкатенацию строк, но это неудобно и громоздко.

Синтаксис функции выглядит следующим образом:

**format(formatstr text [, argument1 text,argument2 text...])**  

где *formatstr* — это шаблон, который мы передаём. Это обычная строка, в которой указаны места для подстановки аргумента.

Допустим, у нас есть шаблон "Hello, #Имя пользователя#!" и таблица водителей, которым нужно вывести приветствие. Теперь мы можем решить её с помощью format():

**SELECT format**('Hello, %s!', d.first_name)  
**FROM** shipping.driver d  

Комбинация символов ***%s*** обозначает, что вместо них будет подставлен один из аргументов, причём в том же порядке, что и в исходном столбце.

Напишем запрос, который описывает содержимое каждой строки в таблице в виде текста.

**SELECT format**('driver_id = %s, first_name = %s, last_name = %s, address = %s, zip_code = %s, phone = %s, city_id = %s',   
   driver_id, first_name, last_name, address, zip_code, phone, city_id)   
**FROM** shipping.driver d  

Мы перечислили в строке семь пропусков (плэйсхолдеров, или мест для подстановки, — %s), передали семь параметров (все столбцы таблицы) и получили шаблон, заполненный значениями для каждой строки. Если в вашем шаблоне присутствует одинарная кавычка, то для удобства можно вместо одинарных кавычек использовать *два_доллара* (два знака доллара):

**SELECT** *два_доллара* some_string with quotes ' *два_доллара*

#### **Задание 6.5**

Давайте подготовим географическую сводку для каждого города. Напишите SQL-запрос, который выведет описание региона в следующем формате:

[*city_name*] is located in [*state*]. There's [*population*] people living there. Its area is [*area*]

Обратите внимание, точку в конце ставить не нужно. Отсортируйте по названию города в алфавитном порядке. Столбец к выдаче — str (сводка). Пример:

*Abilene is located in Texas. There's 115930 people living there. Its area is 105.10*

*Запрос:*

**SELECT format**('%s is located in %s. There'||*два_доллара*'*два_доллара*||'s %s people living there. Its area is %s', c.city_name, c.state, c.population, c.area) str  
**FROM** sql.city c